In [5]:
!ls CPSC477/subtaskA/baseline

transformer_baseline.py


In [2]:
import sys 
import os
sys.path.append(os.path.relpath("CPSC477/subtaskA/baseline"))
from transformer_baseline import test

2024-05-08 21:45:34.933995: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 21:45:34.985280: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 21:45:36.607980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [32]:
import pandas as pd
from transformers import set_seed
import logging
from sklearn.metrics import f1_score, accuracy_score

In [14]:
# Final evaluation script (all models -> all datasets)

id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1}

for dataset in ["subtaskA", "synonym", "synonym2", "homoglyph", "duplicate"]:
    set_seed(0)
    
    #using test sets for final evaluation now; previously set to dev sets
    test_df = pd.read_json(f"CPSC477/subtaskA/data/{dataset}_test.jsonl", lines=True)
    
    for model in ["base", "synonym", "synonym2", "homoglyph", "duplicate"]:
        results, predictions = test(test_df, f"{model}-roberta/subtaskA/0/best/", id2label, label2id)
            
        logging.info(results)
        predictions_df = pd.DataFrame({'id': test_df['id'], 'label': predictions})
        predictions_df.to_json(f"CPSC477/subtaskA/pred/{model}_on_{dataset}.jsonl", lines=True, orient='records')
        print("Evaluated " + model + " on " + dataset)

Map:   0%|          | 0/34272 [00:00<?, ? examples/s]

/home/cpsc477_jhy24/.conda/envs/cs477/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluated base on synonym2


Map:   0%|          | 0/34272 [00:00<?, ? examples/s]

/home/cpsc477_jhy24/.conda/envs/cs477/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

/home/cpsc477_jhy24/.conda/envs/cs477/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluated synonym on duplicate


Map:   0%|          | 0/34272 [00:00<?, ? examples/s]

/home/cpsc477_jhy24/.conda/envs/cs477/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluated homoglyph on duplicate


In [41]:
# Ensemble code

for dataset in ["subtaskA", "synonym", "synonym2", "homoglyph", "duplicate"]:
    first_file = pd.read_json(f"CPSC477/subtaskA/pred/base_on_{dataset}.jsonl", lines=True)[['id', 'label']]
    second_file = pd.read_json(f"CPSC477/subtaskA/pred/synonym_on_{dataset}.jsonl", lines=True)[['id', 'label']]
    third_file = pd.read_json(f"CPSC477/subtaskA/pred/synonym2_on_{dataset}.jsonl", lines=True)[['id', 'label']]
    fourth_file = pd.read_json(f"CPSC477/subtaskA/pred/homoglyph_on_{dataset}.jsonl", lines=True)[['id', 'label']]
    fifth_file = pd.read_json(f"CPSC477/subtaskA/pred/duplicate_on_{dataset}.jsonl", lines=True)[['id', 'label']]
    
    pred_labels = pd.concat([first_file, second_file, third_file, fourth_file, fifth_file], axis=0)
    pred_labels = pred_labels.groupby(['id']).mean()
    pred_labels = pred_labels.round(0).astype(int)

    gold_labels = pd.read_json("CPSC477/subtaskA/data/subtaskA_test.jsonl", lines=True)[['id', 'label']]
    
    merged_df = pred_labels.merge(gold_labels, on='id', suffixes=('_pred', '_gold'))
    

    accuracy = accuracy_score(merged_df['label_gold'], merged_df['label_pred'])

    print(dataset)
    print(accuracy)

subtaskA
0.8418825863678805
synonym
0.8911356209150327
synonym2
0.884016106442577
homoglyph
0.559640522875817
duplicate
0.8602941176470589
